In [15]:
import pandas as pd
import yfinance as yf
import ccxt
from datetime import datetime, timezone, timedelta
from mom import influxDB_utils as influx
from mom import Mandelbrot


In [19]:
# Config
ASSETS = ["BTC", "ETH"]
CURRENCIES = ["EUR"]
GRANULARITIES = {
    "Day": "1d",
    "Hour": "1h",
    "Minute": "1m",
    # "Week": not directly supported in yfinance → use 1d and resample
}

In [20]:
client = influx.get_client()
query_api = client.query_api()

In [29]:
def get_last_timestamp(measurement: str, asset: str, currency: str) -> datetime:
    """
    Query InfluxDB to get the most recent timestamp for a measurement/asset/currency.
    """
    query = f"""
    from(bucket: "{influx.INFLUX_BUCKET}")
      |> range(start: 0)
      |> filter(fn: (r) => r._measurement == "{measurement}")
      |> filter(fn: (r) => r.asset == "{asset}")
      |> filter(fn: (r) => r.currency == "{currency}")
      |> filter(fn: (r) => r._field == "high")
      |> last()
    """
    tables = query_api.query(org=influx.INFLUX_ORG, query=query)
    if not tables or not tables[0].records:
        return None
    return tables[0].records[0].get_time()


def fetch_data(asset: str,currency: str, interval: str, start: datetime) -> pd.DataFrame:
    """
    Fetch OHLC data from yfinance from 'start' until now.
    """
    ticker = yf.Ticker(f"{asset}-{currency}")
    print(GRANULARITIES.get(interval))
    df = ticker.history(interval=GRANULARITIES.get(interval), start=start)
    if df.empty:
        return df
    
    df = Mandelbrot.reset_index(df)
    df = Mandelbrot.process_raw_DF(df)
    df = df.set_index("date")
    df["asset"] = asset
    df["currency"] = currency
    df["interval"] = interval
    for i in ["volume", "high","low","delta","delta_log","return","return_log"]:
        df[i] = df[i].astype(float)
    #print(df.head())
    return df


def update():
    for curr in CURRENCIES:
        for asset in ASSETS:
            for interval, gran in GRANULARITIES.items():
                measurement = f"{interval}"
                print(gran, interval)
                last_time = get_last_timestamp(measurement, asset, curr)
                start = last_time + timedelta(seconds=1) if last_time else datetime(2015, 1, 1, tzinfo=timezone.utc)

                print(f"Updating {measurement} for {asset} since {start}")

                df = fetch_data(asset,curr , interval, start)
                if df.empty:
                    print(f"No new data for {asset} ({interval})")
                    continue

                influx.write_dataframe(df)
                print(f"Updated {asset} ({interval}) with {len(df)} new rows")


if __name__ == "__main__":
    update()

1d Day
Updating Day for BTC since 2025-08-29 00:00:01+00:00
1d
Wrote 3 rows to InfluxDB bucket 'CryptoPrices'
Updated BTC (Day) with 3 new rows
1h Hour
Updating Hour for BTC since 2025-08-29 00:00:01+00:00
1h
Wrote 60 rows to InfluxDB bucket 'CryptoPrices'
Updated BTC (Hour) with 60 new rows
1m Minute
Updating Minute for BTC since 2025-08-29 00:00:01+00:00
1m
Wrote 3090 rows to InfluxDB bucket 'CryptoPrices'
Updated BTC (Minute) with 3090 new rows
1d Day
Updating Day for ETH since 2025-08-29 00:00:01+00:00
1d
Wrote 3 rows to InfluxDB bucket 'CryptoPrices'
Updated ETH (Day) with 3 new rows
1h Hour
Updating Hour for ETH since 2025-08-29 00:00:01+00:00
1h
Wrote 60 rows to InfluxDB bucket 'CryptoPrices'
Updated ETH (Hour) with 60 new rows
1m Minute
Updating Minute for ETH since 2025-08-29 00:00:01+00:00
1m
Wrote 3090 rows to InfluxDB bucket 'CryptoPrices'
Updated ETH (Minute) with 3090 new rows


In [22]:
GRANULARITIES.get(interval)

NameError: name 'interval' is not defined

In [30]:
last_time = get_last_timestamp("Day", "BTC", "EUR")
print(last_time)

2025-08-31 00:00:00+00:00


In [6]:
start = last_time + timedelta(seconds=1) if last_time else datetime(2015, 1, 1, tzinfo=timezone.utc)
print(start)

2025-08-29 00:00:01+00:00


In [16]:
df = fetch_data("BTC","EUR","Day", start)

Index(['unix', 'volume', 'high', 'low', 'delta', 'delta_log', 'return',
       'return_log'],
      dtype='object')
                  unix       volume          high           low        delta  \
date                                                                           
2025-08-29  1756425600  66615328407  96446.390625  92045.328125          NaN   
2025-08-30  1756512000  44054324942  93201.585938  91946.765625 -3244.804688   
2025-08-31  1756598400  38736064512  93657.445312  92745.210938   455.859375   

            delta_log    return  return_log asset currency interval  
date                                                                 
2025-08-29        NaN       NaN         NaN   BTC      EUR      Day  
2025-08-30  -3.511189 -0.033644   -0.033644   BTC      EUR      Day  
2025-08-31   2.658831  0.004891    0.004891   BTC      EUR      Day  


In [14]:
df.head()

,unix,date,volume,high,low,delta,delta_log,return,return_log,asset,currency,interval
0,1756425600,2025-08-29,66615328407,96446.390625,92045.328125,NaN,NaN,NaN,NaN,BTC,EUR,Day
1,1756512000,2025-08-30,44054324942,93201.585938,91946.765625,-3244.804688,-3.511189,-0.033644,-0.033644,BTC,EUR,Day
2,1756598400,2025-08-31,38760181760,93657.445312,92745.210938,455.859375,2.658831,0.004891,0.004891,BTC,EUR,Day


In [17]:
ticker = yf.Ticker("BTC")
df = ticker.history(interval="1d", start=start)

In [15]:
x = ["high_x","high_y","high"]

if "high_x" in x:
    print("yes")
else:
    print("no")

yes


In [46]:
import requests
import pandas as pd
import time
from datetime import datetime

# Binance Kline endpoint
BASE_URL = "https://api.binance.com/api/v3/klines"

def fetch_klines(symbol="BTCEUR", interval="1m", start_time=None, end_time=None, limit=1000):
    """
    Fetch candlestick data from Binance API.
    """
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    if start_time:
        params["startTime"] = start_time
    if end_time:
        params["endTime"] = end_time

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if "code" in data:  # error
        raise Exception(data)

    return data

def get_historical_klines(symbol="BTCEUR", interval="1m", start_date="2022-07-01", save_csv="BTC_EUR_m_2024.csv"):
    """
    Download all historical klines from start_date until now.
    """
    end = "2022-12-31"
    # convert start date to milliseconds
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    end_time = int(datetime.strptime(end, "%Y-%m-%d").timestamp() * 1000)
    #end_time = int(time.time() * 1000)

    all_data = []
    while start_time < end_time:
        klines = fetch_klines(symbol, interval, start_time=start_time, limit=1000)
        if not klines:
            break

        all_data.extend(klines)

        # move to next batch (last candle's close time + 1 ms)
        start_time = klines[-1][6] + 1  

        print(f"Fetched {len(all_data)} rows so far...")

        time.sleep(0.2)  # avoid hitting API rate limits

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "num_trades",
        "taker_buy_base", "taker_buy_quote", "ignore"
    ])

    # Convert timestamp to datetime
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")

    # Save CSV
    df.to_csv(save_csv, index=False)
    print(f"Saved {len(df)} rows to {save_csv}")

    return df



In [47]:
df = get_historical_klines()

Fetched 1000 rows so far...
Fetched 2000 rows so far...
Fetched 3000 rows so far...
Fetched 4000 rows so far...
Fetched 5000 rows so far...
Fetched 6000 rows so far...
Fetched 7000 rows so far...
Fetched 8000 rows so far...
Fetched 9000 rows so far...
Fetched 10000 rows so far...
Fetched 11000 rows so far...
Fetched 12000 rows so far...
Fetched 13000 rows so far...
Fetched 14000 rows so far...
Fetched 15000 rows so far...
Fetched 16000 rows so far...
Fetched 17000 rows so far...
Fetched 18000 rows so far...
Fetched 19000 rows so far...
Fetched 20000 rows so far...
Fetched 21000 rows so far...
Fetched 22000 rows so far...
Fetched 23000 rows so far...
Fetched 24000 rows so far...
Fetched 25000 rows so far...
Fetched 26000 rows so far...
Fetched 27000 rows so far...
Fetched 28000 rows so far...
Fetched 29000 rows so far...
Fetched 30000 rows so far...
Fetched 31000 rows so far...
Fetched 32000 rows so far...
Fetched 33000 rows so far...
Fetched 34000 rows so far...
Fetched 35000 rows so f

In [5]:
df.head()
#df.to_csv("BTC_EUR_m_2022-2025.csv")

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base,taker_buy_quote,ignore
0,2021-12-31 23:00:00,40698.13000000,40751.92000000,40693.92000000,40745.48000000,0.87253000,2021-12-31 23:00:59.999,35531.84492750,25,0.83561000,34027.51112390,0
1,2021-12-31 23:01:00,40752.66000000,40763.74000000,40752.66000000,40763.74000000,0.73071000,2021-12-31 23:01:59.999,29782.96929480,14,0.73071000,29782.96929480,0
2,2021-12-31 23:02:00,40771.00000000,40785.53000000,40770.86000000,40785.53000000,0.28062000,2021-12-31 23:02:59.999,11442.49956920,6,0.25855000,10542.68575660,0
3,2021-12-31 23:03:00,40785.53000000,40794.58000000,40781.93000000,40781.93000000,0.04652000,2021-12-31 23:03:59.999,1897.23545110,7,0.01326000,540.82490630,0
4,2021-12-31 23:04:00,40781.93000000,40800.77000000,40780.20000000,40795.85000000,0.07778000,2021-12-31 23:04:59.999,3173.29443160,16,0.06675000,2723.38496770,0


In [40]:
DF.dtypes

high          float64
volume        float64
asset          object
currency       object
delta         float64
delta_log     float64
return        float64
return_log    float64
interval       object
dtype: object

In [48]:
DF = df
DF.columns = map(str.lower, DF.columns)
DF["currency"] = "EUR"
DF["asset"] = "BTC"
#delta etc
DF["high"] = DF["high"].astype(float)
DF["delta"] = DF["high"].diff()
DF["delta_log"] = Mandelbrot.log10(DF["delta"])
DF["return"] = Mandelbrot.compute_returns(DF["high"])[0]
DF["return_log"] = Mandelbrot.compute_returns(DF["high"])[1]

DF['date'] = pd.to_datetime(DF['open_time'], unit='s')
DF = DF.set_index("date")
DF["interval"] = "Minute"
co =['high','volume','asset', 'currency', 'delta', 'delta_log', 'return','return_log', 'interval']
DF = DF[co]

 # Ensure numeric columns are floats
for col in ["high","low","volume", "delta", "delta_log", "return", "return_log"]:
    if col in DF.columns:
        DF[col] = pd.to_numeric(DF[col], errors="coerce")
# Ensure tag columns are strings
for col in ["asset", "currency", "interval"]:
    if col in DF.columns:
        DF[col] = DF[col].astype(str)



DF.head()


,high,volume,asset,currency,delta,delta_log,return,return_log,interval
date,,,,,,,,,
2022-06-30 22:00:00,17993.30,0.50952,BTC,EUR,NaN,NaN,NaN,NaN,Minute
2022-06-30 22:01:00,17983.89,0.30962,BTC,EUR,-9.41,-0.973590,-0.000523,-0.000523,Minute
2022-06-30 22:02:00,17998.73,0.69929,BTC,EUR,14.84,1.171434,0.000825,0.000825,Minute
2022-06-30 22:03:00,17975.27,0.17472,BTC,EUR,-23.46,-1.370328,-0.001303,-0.001304,Minute
2022-06-30 22:04:00,17982.35,1.28754,BTC,EUR,7.08,0.850033,0.000394,0.000394,Minute


In [34]:
DF.head()

,high,volume,asset,currency,delta,delta_log,return,return_log,interval
date,,,,,,,,,
2024-12-31 23:00:00,88795.64,0.18342,BTC,EUR,NaN,NaN,NaN,NaN,Minute
2024-12-31 23:01:00,89013.48,0.75052,BTC,EUR,217.84,2.338138,0.002453,0.002450,Minute
2024-12-31 23:02:00,89203.20,0.09794,BTC,EUR,189.72,2.278113,0.002131,0.002129,Minute
2024-12-31 23:03:00,89756.53,4.63110,BTC,EUR,553.33,2.742984,0.006203,0.006184,Minute
2024-12-31 23:04:00,89788.80,1.53803,BTC,EUR,32.27,1.508799,0.000360,0.000359,Minute


In [49]:
influx.write_dataframe(DF, measurement="Minute")

Wrote 264000 rows to InfluxDB bucket 'CryptoPrices'


In [ ]:
#combine with csv data



def old_csv(c,g,year,interv):

        
    DF = pd.read_csv(f"historical_data/{year}_{g}_{c}_to_EUR.csv")
    DF.columns = map(str.lower, DF.columns)
    DF["currency"] = DF["curr"]
    #delta etc
    DF["delta"] = DF["high"].diff()
    DF["delta_log"] = Mandelbrot.log10(DF["delta"])
    DF["return"] = Mandelbrot.compute_returns(DF["high"])[0]
    DF["return_log"] = Mandelbrot.compute_returns(DF["high"])[1]

    #datetype M
    DF['date'] = pd.to_datetime(DF['unix timestamp'], unit='s')
    rfc3339 = []
    for i in DF["unix timestamp"]:
        dt = datetime.fromtimestamp(i, tz= timezone.utc)
        date = dt.isoformat().replace("+00:00","Z")
        rfc3339.append(date)
    DF["date_rfc"] = rfc3339

    DF = DF.set_index("date")
    DF["interval"] = interv
    co =['high','volume','asset', 'currency', 'delta', 'delta_log', 'return',
           'return_log', 'interval']
    DF = DF[co]

    # Ensure numeric columns are floats
    for col in ["high","low","volume", "delta", "delta_log", "return", "return_log"]:
        if col in DF.columns:
            DF[col] = pd.to_numeric(DF[col], errors="coerce")

    # Ensure tag columns are strings
for col in ["asset", "currency", "interval"]:
    if col in DF.columns:
        DF[col] = DF[col].astype(str)


DF.index = pd.to_datetime(DF.index, utc=True